In [1]:
import os
import openai
import pathlib
import wave
from datetime import datetime
import chunked_tts
import base64
import text_extraction

/Users/Itai/Library/Mobile Documents/com~apple~CloudDocs/independent_research/paper2audio/.venv/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
FILEPATH = pathlib.Path('/Users/Itai/Downloads/s41586-025-09697-2_reference.pdf')
PPMT = 12 # price per million tokens for audio generation in dollars. Used to set a price cap for the audio generation.
PRICE_CAP = 0.5 # price cap for the audio generation in dollars.
OUTPUT_FILE = f'ezspecificity_{datetime.now().strftime("%Y%m%d_%H%M%S")}'
FORMAT = "mp3"
TEXT_MODEL = "gpt-4.1-mini-2025-04-14"
TTS_MODEL = "gpt-4o-mini-tts"
client = openai.OpenAI()
speech_file_path = pathlib.Path(OUTPUT_FILE)

In [ ]:
# Extract (or load cached) narratable text
extracted_text = text_extraction.extract_text_from_pdf(
    FILEPATH,
    OUTPUT_FILE,
    prompt=None,  # use default prompt in library, or supply custom
    model=TEXT_MODEL,
    client=client,
)

print(extracted_text[:2000]+'... [TRUNCATED]')  # preview first 2k chars

# continue workflow
parsed_response = type("Obj", (), {"choices": [type("msg", (), {"message": type("m", (), {"content": extracted_text})()})]})()

Saved extracted text to ezspecificity_20251014_161545.txt
This is an automated voice reading Enzyme specificity prediction using cross attention graph neural networks by Haiyang Cui et al.

nature

Accelerated Article Preview

Enzyme specificity prediction using cross attention graph neural networks

Haiyang Cui, Yufeng Su, Tanner J. Dean, Tianhao Yu, Zhengyi Zhang, Jian Peng, Diwakar Shukla & Huimin Zhao

Enzymes are the molecular machines of life, and a key property that governs their function is substrate specificity—the ability of an enzyme to recognize and selectively act on particular substrates. This specificity originates from the three-dimensional structure of the enzyme active site and complicated transition state of the reaction. Many enzymes can promiscuously catalyze reactions or act on substrates beyond those for which they were originally evolved. However, millions of known enzymes still lack reliable substrate specificity information, impeding their practical applicatio

In [4]:
# Estimate the cost of the TTS (may be very inaccurate)

import tiktoken 

tokenizer = tiktoken.encoding_for_model("gpt-4o-mini")
encoded = tokenizer.encode (parsed_response.choices[0].message.content)
print (len(encoded))
cost_estimate = PPMT*len(encoded)/1_000_000
print(f'estimated cost = {cost_estimate}$')

4995
estimated cost = 0.05994$


In [5]:
### Initialize block that can be run independently of the block below if it igets interrupted
chunks_processed = 0
pcm_all = b""

In [6]:
if cost_estimate < PRICE_CAP:
   chunks = chunked_tts.chunk_text_by_lines(parsed_response.choices[0].message.content)
   print(f"Splitting input into {len(chunks)} chunks…")
   # try retrieving progress
   # Check if idx is defined, and if so, start from chunks[idx:]
   
if chunks_processed > 0:
    print(f"Resuming from chunk {chunks_processed}/{len(chunks)}")
    chunks = chunks[chunks_processed:]  # idx is 1-based, so subtract 1 for 0-based indexing

for chunk in chunks:
    print(f"→ Processing chunk {chunks_processed+1}/{len(chunks)} (chars={len(chunk)})")
    pcm_all += chunked_tts.tts_chunk(client, chunk)
    chunks_processed += 1

Splitting input into 4 chunks…
→ Processing chunk 1/4 (chars=7720)
Completed TTS chunk
→ Processing chunk 2/4 (chars=7857)
Completed TTS chunk
→ Processing chunk 3/4 (chars=7702)
Completed TTS chunk
→ Processing chunk 4/4 (chars=4122)
Completed TTS chunk


In [7]:
chunked_tts.save_audio_file(OUTPUT_FILE, pcm_all, fmt=FORMAT)
print(f"Wrote {OUTPUT_FILE} (size={len(pcm_all)/24000/2:.1f} s)")      

FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'

In [8]:
chunks

['This is an automated voice reading Complete computational design of high-efficiency Kemp elimination enzymes by Dina Listov et al.\n\nComplete computational design of high-efficiency Kemp elimination enzymes\n\nUntil now, computationally designed enzymes exhibited low catalytic rates and required intensive experimental optimization to reach activity levels observed in comparable natural enzymes. These results exposed limitations in design methodology and suggested critical gaps in our understanding of the fundamentals of biocatalysis. We present a fully computational workflow for designing efficient enzymes in TIM-barrel folds using backbone fragments from natural proteins and without requiring optimization by mutant-library screening. Three Kemp eliminase designs exhibit efficiencies greater than 2,000 M inverse seconds. The most efficient shows more than 140 mutations from any natural protein, including a novel active site. It exhibits high stability greater than 85 degrees Celsius

3